In [1]:
import requests
import pandas as pd
import pymongo
import datetime
from dateutil import parser
import concurrent.futures
import statistics
import math


In [2]:
df = pd.read_csv('nifty_500.csv')

In [3]:
client = pymongo.MongoClient('mongodb://127.0.0.1:27017/')
mydb = client['Algorithmic_Trading']
info = mydb.historical_data_500

In [4]:
headers = {'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.131 Safari/537.36'}

In [5]:
stocks_used = df['Symbol'].tolist()

In [6]:
urls = []
for s in stocks_used:
    url = f"https://query1.finance.yahoo.com/v8/finance/chart/{s}.NS?formatted=true&crumb=M6Y926IjM5W&lang=en-US&region=US&includeAdjustedClose=true&interval=1d&period1=724291200&period2=1635811200&events=capitalGain%7Cdiv%7Csplit&useYfid=true&corsDomain=finance.yahoo.com"
    urls.append(url)

In [7]:

def append_it(u):
    
    r = requests.get(u , headers = headers)
    data = r.json()
    
    
    
    Time = data['chart']['result'][0]['timestamp']
    Low = data['chart']['result'][0]['indicators']['quote'][0]['low']
    High = data['chart']['result'][0]['indicators']['quote'][0]['high']
    Open = data['chart']['result'][0]['indicators']['quote'][0]['open']
    Close = data['chart']['result'][0]['indicators']['quote'][0]['close']
    Volume = data['chart']['result'][0]['indicators']['quote'][0]['volume']
    
    total_length = len(Time)
    
    hist_data = []
    
    for i in range(0,total_length):
        
        
        db_dict = {}
        
        single = {}
        
        
        Date = Time[i]
        
        r = datetime.datetime.fromtimestamp(Date)
        
        month = r.month
        if(r.month < 10):
            month = '0' + str(r.month)

        day = r.day
        if(r.day < 10):
            day = '0' + str(r.day)
            
        year = r.year

        single['Date'] = str(year) + '-' + str(month) + '-' + str(day)
        
        if Open[i] is not None:
            single['Open'] = Open[i]
        else:
            Open_val = Open[i-1]
            Open[i] = Open_val
            
            single['Open'] = Open[i]
            
            
            
        if High[i] is not None:
            single['High'] = High[i]
        else:
            High_val = High[i-1]
            High[i] = High_val
            
            single['High'] = High[i]
            
            
            
        if Low[i] is not None:
            single['Low'] = Low[i]
        else:
            Low_val = Low[i-1]
            Low[i] = Low_val
            
            single['Low'] = Low[i]

            
            
        if Close[i] is not None:
            single['Close'] = Close[i]
        else:
            Close_val = Close[i-1]
            Close[i] = Close_val
            
            single['Close'] = Close[i]
            
            
            
        if Volume[i] is not None:
            single['Volume'] = Volume[i]
        else:
            Volume_val = Volume[i-1]
            Volume[i] = Volume_val
        
            single['Volume'] = Volume[i]
            
            
        
        hist_data.append(single)
    db_dict['Symbol'] = u.rpartition('chart/')[2].rpartition('.NS')[0]
    db_dict['Data'] = hist_data
    
    
    info.insert_one(db_dict)
    
        


In [8]:
with concurrent.futures.ThreadPoolExecutor(max_workers = 20) as executor:
    executor.map(append_it,urls)